In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms
from net import *
from utils import *
from loss import *

In [2]:
def quantitative_eval(model, dataset, parameters, target_mask):
    
    ## parameters
    device = parameters['device']
    batch_size = parameters['batch_size']
    chunk_size = parameters['chunk_size']
    all_time_max = parameters['all_time_max']
    
    ## metrics
    hole_l1_output = []
    hole_mse_output = []
    hole_count = []
    hole_total_pred = []
    hole_total_gt = []
    
    ## iterate through
    outer_indices = [k for k in range(chunk_size-1, len(dataset), chunk_size)]
    for i in range(0, len(outer_indices), batch_size):
        inner_indices = range(i, min(len(outer_indices), i+batch_size))
        _, gt= zip(*[dataset[outer_indices[j]] for j in inner_indices])
        gt = torch.stack(gt).to(device)
        mask = np.repeat(target_mask.reshape(1,64,64), chunk_size, axis=0).reshape(1,chunk_size,64,64)
        mask = torch.from_numpy(np.repeat(mask, gt.shape[0], axis=0).reshape(gt.shape[0],1,chunk_size,64,64)).float().to(device)
        with torch.no_grad():
            output, _ = model(gt, mask)
        output_comp = mask * gt + (1 - mask) * output
        
        ## scale back
        gt = (gt*all_time_max).squeeze_(1).cpu().detach().numpy()
        output_comp = (output_comp*all_time_max).squeeze_(1).cpu().detach().numpy()
        mask = mask.squeeze_(1).cpu().detach().numpy()      
        
        for j in range(len(output)):
            for k in range(chunk_size):
                ## single image & output
                gt_single = gt[j][k]
                output_comp_single = output_comp[j][k]
                mask_single = mask[j][k]
            
                ## hole regions
                output_comp_single_hole = output_comp_single[np.where(mask_single == 0)]
                gt_single_hole = gt_single[np.where(mask_single == 0)]
                hole_l1_output.append(np.mean(np.abs(output_comp_single_hole - gt_single_hole)))
                hole_mse_output.append(mean_squared_error(output_comp_single_hole, gt_single_hole))
                
                hole_count.append(np.sum(mask_single==0))
                hole_total_pred.append(np.sum(output_comp_single_hole))
                hole_total_gt.append(np.sum(gt_single_hole))
            
    return {'hole_l1_output':hole_l1_output,
            'hole_mse_output':hole_mse_output,
            'hole_count':hole_count,
            'hole_total_pred':hole_total_pred,
            'hole_total_gt':hole_total_gt}

### Urban Masking

In [3]:
parameters = {
    "batch_size": 16,
    "chunk_size": 2,
    "all_time_max": 1428,
    "device": torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
}
results = {}
img = np.zeros((64,64))
img = np.load(f'D:/nyc_taxi/data_min_max/train.npy')[0]
blurred = cv2.GaussianBlur(img, (5,5), 0)
urban_mask = 1-cv2.threshold(blurred, np.quantile(blurred.ravel(), 0.9), 1, cv2.THRESH_BINARY)[1]

for chunk_size in [1,2,3,5,7,10,15]:
    print(f'Chunk Size: {chunk_size}')
    for mask_type in ['biased', 'random']:
        for mape_loss in [0, 1]:
            parameters['chunk_size'] = chunk_size
            parameters['mape_loss'] = mape_loss
            ## parameters
            img_root = "D:/nyc_taxi/data_min_max"
            mask_root = "D:/nyc_taxi/data_min_max"
            image_size = 64
            device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
            test_imgs = np.load(img_root+'/test.npy')
            test_masks_random = np.load(mask_root+'/test_random_mask.npy')
            dataset_test = taxi_data(test_imgs, test_masks_random, image_size, chunk_size)

            ## load models
            model = PConvUNet(chunk_size=chunk_size).to(device)
            model.load_state_dict(torch.load(f'../../model_states/chunk_size_{chunk_size}/{mask_type}_{mape_loss}'))
            model.eval()
            
            ## evaluation
            test_result = quantitative_eval(model, dataset_test, parameters, urban_mask)
            results[f'{mask_type}_{mape_loss}_{chunk_size}'] = test_result

Chunk Size: 1
Chunk Size: 2
Chunk Size: 3
Chunk Size: 5
Chunk Size: 7
Chunk Size: 10
Chunk Size: 15


In [4]:
random_tabular_view = []

for key in results.keys():
    
    result = results[key]
    
    ## random test set evaluation
    random_tabular_view.append([
        key,
        np.mean(result['hole_l1_output']),
        np.mean(result['hole_mse_output']),
        np.mean(result['hole_count']),
        np.mean(result['hole_total_pred']),
        np.mean(result['hole_total_gt'])
    ])
## make tabular view
random_tabular_view = pd.DataFrame(random_tabular_view, columns=['model', 'hole_l1_output', 'hole_mse_output', 
                                                                 'hole_count', 'hole_total_pred', 'hole_total_gt'])
random_tabular_view

,model,hole_l1_output,hole_mse_output,hole_count,hole_total_pred,hole_total_gt
0,biased_0_1,18.315262,1226.642619,409.0,14903.681641,14569.634766
1,biased_1_1,18.517277,1185.115405,409.0,15952.291016,14569.634766
2,random_0_1,20.047932,1537.730345,409.0,10324.297852,14569.634766
3,random_1_1,20.311567,1592.321249,409.0,9857.490234,14569.634766
4,biased_0_2,18.917740,1266.970676,409.0,15935.589844,14569.634766
5,biased_1_2,18.323633,1199.310320,409.0,11800.934570,14569.634766
6,random_0_2,20.059307,1500.263831,409.0,13133.112305,14569.634766
7,random_1_2,20.970039,1681.684728,409.0,9247.792969,14569.634766
8,biased_0_3,20.984777,1655.356508,409.0,9608.748047,14569.634766
9,biased_1_3,19.153976,1343.842238,409.0,11278.986328,14569.634766


In [5]:
random_tabular_view.to_csv(f'urban_target_results.csv', index=False)

### Rural Eval

In [6]:
parameters = {
    "batch_size": 16,
    "chunk_size": 2,
    "all_time_max": 1428,
    "device": torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
}
results = {}
img = np.zeros((64,64))
img = np.load(f'D:/nyc_taxi/data_min_max/train.npy')[0]
blurred = cv2.GaussianBlur(img, (5,5), 0)
rural_mask = cv2.threshold(blurred, np.quantile(blurred.ravel(), 0.9), 1, cv2.THRESH_BINARY)[1]

for chunk_size in [1,2,3,5,7,10,15]:
    print(f'Chunk Size: {chunk_size}')
    for mask_type in ['biased', 'random']:
        for mape_loss in [0, 1]:
            parameters['chunk_size'] = chunk_size
            parameters['mape_loss'] = mape_loss
            ## parameters
            img_root = "D:/nyc_taxi/data_min_max"
            mask_root = "D:/nyc_taxi/data_min_max"
            image_size = 64
            device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
            test_imgs = np.load(img_root+'/test.npy')
            test_masks_random = np.load(mask_root+'/test_random_mask.npy')
            dataset_test = taxi_data(test_imgs, test_masks_random, image_size, chunk_size)

            ## load models
            model = PConvUNet(chunk_size=chunk_size).to(device)
            model.load_state_dict(torch.load(f'../../model_states/chunk_size_{chunk_size}/{mask_type}_{mape_loss}'))
            model.eval()
            
            ## evaluation
            test_result = quantitative_eval(model, dataset_test, parameters, rural_mask)
            results[f'{mask_type}_{mape_loss}_{chunk_size}'] = test_result

Chunk Size: 1
Chunk Size: 2
Chunk Size: 3
Chunk Size: 5
Chunk Size: 7
Chunk Size: 10
Chunk Size: 15


In [7]:
random_tabular_view = []

for key in results.keys():
    
    result = results[key]
    
    ## random test set evaluation
    random_tabular_view.append([
        key,
        np.mean(result['hole_l1_output']),
        np.mean(result['hole_mse_output']),
        np.mean(result['hole_count']),
        np.mean(result['hole_total_pred']),
        np.mean(result['hole_total_gt'])
    ])
## make tabular view
random_tabular_view = pd.DataFrame(random_tabular_view, columns=['model', 'hole_l1_output', 'hole_mse_output', 
                                                                 'hole_count', 'hole_total_pred', 'hole_total_gt'])
random_tabular_view

,model,hole_l1_output,hole_mse_output,hole_count,hole_total_pred,hole_total_gt
0,biased_0_1,0.301297,6.681198,3687.0,814.548767,552.103455
1,biased_1_1,0.192829,2.351459,3687.0,590.820129,552.103455
2,random_0_1,0.865638,27.720801,3687.0,2817.428711,552.103455
3,random_1_1,0.226243,3.568071,3687.0,738.920654,552.103455
4,biased_0_2,0.459805,10.879715,3687.0,1595.531982,552.103455
5,biased_1_2,0.135609,2.051972,3687.0,492.403900,552.103455
6,random_0_2,0.517540,11.431809,3687.0,1827.294678,552.103455
7,random_1_2,0.107706,1.018128,3687.0,341.364502,552.103455
8,biased_0_3,0.142266,1.926650,3687.0,423.703857,552.103455
9,biased_1_3,0.102333,0.918139,3687.0,348.199921,552.103455


In [8]:
random_tabular_view.to_csv(f'rural_target_results.csv', index=False)